## 2022 Yellowstone National Park Flood

In June 2022, parts of Montana and Wyoming experienced an extreme rain
event. 2 to 3 inches of rain fell in a short period which caused a large
amount of snow to melt. The resulting runoff caused massive flooding
throughout Yellowstone National Park. Multiple park entrances were
closed and park infrastructure, particularly roads and bridge, sustained
high levels of damage.

Climatology: The more arid northern regions of YNP receive approximately
15 to 20 inches of rain per year. Average summer daytime high temperatures
range from 70°F to 80°F

Ecology: Yellowstone National Park is known for it's diverse and pristine landscapes, including rugged mountains, high desert, riverine valleys, and mixed conifer forests. Yellowstone is home to many species of plants and wildlife but is most well known for large populations of bison, elk, pronghorn, bighorn sheep, mule deer, and coyotes. Wolves and grizzly bears are also present here.

Communities and Infrastructure: Yellowstone National Park is largely undeveloped but a network of roads, campgrounds, interpretive centers, hotels, and other recreational facilities are present.

In [1]:
import subprocess
import warnings

from io import BytesIO

import folium
import hvplot.pandas
import hydrofunctions as hf
import holoviews as hv
from holoviews import dim
import pandas as pd
import requests

hv.extension('matplotlib')

hv.extension('bokeh')

warnings.filterwarnings('ignore')

### Map showing the location of the Lamar River USGS Stream Gauge

In [2]:
sg_lat =  44.92817778 
sg_lon =  -110.3942694 

# Initialize map and tweak settings
folium.Figure(width=1000, height=1000)
m = folium.Map(
    # Location to display
    location=(sg_lat, sg_lon),
    # Turns off annoying zooming while trying to scroll to the next cell
    scrollWheelZoom=False,
    tiles = "https://server.arcgisonline.com/ArcGIS/rest/services/NatGeo_World_Map/MapServer/tile/{z}/{y}/{x}",
    attr = 'Tiles &copy; Esri &mdash; National Geographic, Esri, DeLorme, NAVTEQ, UNEP-WCMC, USGS, NASA, ESA, METI, NRCAN, GEBCO, NOAA, iPC',
    zoom_start = 9
)

# Put a marker at the stream gauge location
folium.Marker([sg_lat, sg_lon], popup="Lamar River USGS Gauge, Yellowstone NP").add_to(m)

# Display the map
m

In [3]:
# URL for stream flow data for Lamar River USGS gauge

ynp_usgs_url = ("https://waterdata.usgs.gov/nwis/dv?cb_00060=on&format="
            "rdb&site_no=06188000&legacy=&referred_module=sw&period="
            "&begin_date=1923-10-01&end_date=2023-09-17")

ynp_usgs_url

ynp_response = requests.get(ynp_usgs_url)

## The following plot contains streamflow values for the Lamar River USGS site from 1923 through present.

In [4]:
# parse csv into dataframe for all data

ynp_streamflow = pd.read_csv(
    BytesIO(ynp_response.content),
    comment='#',
    delimiter='\t',
    skiprows=[29, 30],
    names=['Agency', 'Site', 'Date', 'CFS', 'Code'],
    index_col='Date',
    parse_dates=True,
)

ynp_streamflow['Year'] = pd.to_datetime(ynp_streamflow.index, format='%Y%m').strftime('%Y')

ynp_plot_all = ynp_streamflow.hvplot(x='Date', y='CFS',
                                             title='Stream Discharge - Lamar River in Yellowstone National Park, 1923 through Present',
                                             ylabel='Discharge (cubic feet per second)',
                                             xlabel='Date',
                                             yformatter='%.0f',
                                             width=1000
                                             )
ynp_plot_all

:Curve   [Date]   (CFS)

In [5]:
# # make a subset of the data starting in 1989 after a gap in monitoring

# ynp_subset = ynp_streamflow['1989':]

# ynp_subset_plot = ynp_subset.hvplot(x='Date', y='CFS',
#                                              title='Stream Discharge - Lamar River in Yellowstone National Park, 1989 through present',
#                                              ylabel='Discharge (cubic feet per second)',
#                                              xlabel='Date',
#                                              yformatter='%.0f',
#                                              width=1000
#                                              )
# ynp_subset_plot

### The following plot shows maximum annual stream discharge for the Lamar River USGS site in Yellowstone National Park

In [6]:
# resample for maximum values

ynp_resample = ynp_streamflow[['CFS']].resample('AS').max()

ynp_resample_plot = ynp_resample.hvplot(x='Date', y='CFS',
                                             title='Max Annual Stream Discharge - Lamar River in Yellowstone National Park, 1923 through present',
                                             ylabel='Discharge (cubic feet per second)',
                                             xlabel='Date',
                                             yformatter='%.0f',
                                             width=1000
                                             )
ynp_resample_plot

:Curve   [Date]   (CFS)

### The following table shows exceedence propability and flood return period over the 100 years being analyzed.

In [7]:
# calculate exceedance probability and return period

ynp_flood_ranked = (
    ynp_resample.rank(ascending=False)
    / len (ynp_resample)
)

ynp_copy = ynp_resample

ynp_copy['Exceedance_Prob'] = ynp_flood_ranked

ynp_copy['Return_Period'] = (
    1 / ynp_copy.Exceedance_Prob
)

ynp_copy

,CFS,Exceedance_Prob,Return_Period
Date,,,
1923-01-01,218.0,0.831683,1.202381
1924-01-01,4040.0,0.811881,1.231707
1925-01-01,11500.0,0.089109,11.222222
1926-01-01,6230.0,0.653465,1.530303
1927-01-01,11200.0,0.108911,9.181818
...,...,...,...
2019-01-01,9590.0,0.217822,4.590909
2020-01-01,11200.0,0.108911,9.181818
2021-01-01,5960.0,0.673267,1.485294


### The following plot illustrates flood return period for the 100 years analyzed.

In [8]:
ynp_flood_ranked_all = ynp_streamflow[['CFS']].resample('AS').max()

ynp_flood_ranked_all['CFS_rank'] = ynp_flood_ranked_all.rank(ascending=False)

ynp_flood_ranked_all['Exceedance_Prob'] = (
    ynp_flood_ranked_all.CFS_rank
    / len(ynp_flood_ranked_all)
)

ynp_flood_ranked_all['Return_Period'] = (
    1 / ynp_flood_ranked_all.Exceedance_Prob
)

ynp_flood_ranked_all['Year'] = pd.to_datetime(ynp_flood_ranked_all.index, format='%Y%m').strftime('%Y')

# ynp_flood_ranked_all

In [9]:
ynp_analysis_plot = ynp_flood_ranked_all.hvplot.bar(x='Year', y='Return_Period',
                                             title='Flood Return Period - Lamar River in Yellowstone National Park, 1923 through present',
                                             ylabel='Discharge (cubic feet per second)',
                                             xlabel='Date',
                                             yformatter='%.0f',
                                             width=1000,
                                             rot = 90
                                             )

ynp_analysis_plot

:Bars   [Year]   (Return_Period)

### The following plot illustrates the exceedance propability for the 100 years analyzed.

In [10]:
ynp_analysis_plot = ynp_flood_ranked_all.hvplot.bar(x='Year', y='Exceedance_Prob',
                                             title='Exceedance Probability - Lamar River in Yellowstone National Park, 1923 through present',
                                             ylabel='Discharge (cubic feet per second)',
                                             xlabel='Date',
                                             yformatter='%.0f',
                                             width=1000,
                                             rot = 90
                                             )

ynp_analysis_plot

:Bars   [Year]   (Exceedance_Prob)

## Data Source

The data for this project comes from the United States Geologic Survey National Water Dashboard tool. 

## Citation

U.S. Geological Survey, 2023, National Water Information System", data available on the World Wide Web (USGS Water Data for the Nation),accessed September 18th, 2023, at URL https://waterdata.usgs.gov/monitoring-location/06188000/

In [11]:
%%bash
jupyter nbconvert yellowstone_flood.ipynb --to html --no-input

[NbConvertApp] Converting notebook yellowstone_flood.ipynb to html
/opt/conda/share/jupyter/nbconvert/templates/base/display_priority.j2:32: UserWarning: Your element with mimetype(s) dict_keys([]) is not able to be represented.
  {%- elif type == 'text/vnd.mermaid' -%}
[NbConvertApp] Writing 1194502 bytes to yellowstone_flood.html
